# Physical Building by EEP v0.37 - Generate structure for physical building in EEP 
[HOME](https://eepcloud.ngrok.io)

This program lets the user create new physical buildings, add rooms to existing physical buildings and add metadata to existing physical buildings in EEP.
1. Log in to EEP (Choose 'energima' for production and 'energima'dev' for testing purposes
2. Choose create new, add rooms, edit rooms or update metadata
3. Follow the GUI process for each task. The GUI dumps a print of the changes as a final check. If changes are correct, go a head and upload to EEP with the final button (for each task)
4. After finishing the task (and uploading to EEP). A refresh of the page (press HOME) and enter the GUI again, is the safest way to see that all changes have been applied before starting your new task.

***

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime, time, sys


base_path = '/home/mortenhuse/EEP_projects/EEP_DataExtractor_pipeline'

if base_path not in sys.path:
    sys.path.append(base_path)
#print(sys.path)
from eep.client import GenTimeSeriesAPI, PhysicalBuildingAPI

from cognite.client import CogniteClient
from cognite.client.data_classes import AssetUpdate, LabelFilter
from cognite.client.utils import ms_to_datetime

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [2]:
def Convert_strings_to_list_of_floats(string): 
    li = list(string.split(","))
    conv_li = []
    for item in li:
        item = float(item)
        conv_li.append(item)
    return conv_li

def Convert_string_to_tuple(string):
    res = tuple(map(int, string.split(','))) 
    return res

def df_columns_inst_to_list_of_tuples(df, col_name):
    list_of_tuples = []
    col_series = df[str(col_name)]
    col_series = col_series.sort_values(ascending=True)
    for index, val in col_series.iteritems():
        list_of_tuples.append((val, index))
    
    return list_of_tuples

def Convert_semicol_string_to_list_of_tuples(string):
    new_list = []
    #string = txt_period_list.value
    list_string = string.split(';')
    
    for item in list_string:
        
        clean_item = item.replace('(','').replace(')','').replace('','')
        tup_item = tuple(map(int, clean_item.split(',')))
        new_list.append(tup_item)
    
    return new_list

def Convert_col_string_to_list_of_ints(string):
    new_list = []
    #string = txt_period_list.value
    list_string = string.split(',')
    
    for item in list_string:
        
        clean_item = int(item)
        #tup_item = tuple(map(int, clean_item.split(',')))
        new_list.append(clean_item)
    
    return new_list

def get_phys_building(c, active_root_asset_extId, level_str='=2'):
    #print(active_root_asset_extId)
    
    try:
        building = c.assets.retrieve_subtree(external_id = active_root_asset_extId, depth=2).to_pandas()
        found_subbuild = building['externalId'].str.contains(level_str)
        index_match = found_subbuild[found_subbuild].index.values
    except:
        index_match = 0

    if len(index_match) > 0:
        building_prefix = building.iloc[index_match[0]].externalId
        #print(building_prefix)
        AH_phys_build = c.assets.list(root=False, external_id_prefix = building_prefix, limit=-1)
        
    else:
        #print('No physbuild found under rootAsset')
        AH_phys_build = pd.DataFrame()
        
    return AH_phys_build

def find_item_on_level_in_phys_building(c, root_asset, building_level = ['floor']):
    
    """
    Filters out the different parts ("room", "floor","building","sub_building") of a building physical hierarcy
    
    Input:
    root_asset externalId (str)
    filter criterias ('room', 'floor', 'sub_building', 'building') list
    
    Output:
    list of matched externalId
    
    """
        
    my_label_filter = LabelFilter(contains_any=building_level)
    asset_list = c.assets.list(root_external_ids=str(root_asset), labels=my_label_filter, limit=-1)
    return asset_list

def find_item_on_level_in_phys_building_AAA(c, AAA_asset, building_level = ['floor']):
    
    """
    Filters out the different parts ("room", "floor","building","sub_building") of a building physical hierarcy
    
    Input:
    root_asset externalId (str)
    filter criterias ('room', 'floor', 'sub_building', 'building') list
    
    Output:
    list of matched externalId
    
    """
        
    my_label_filter = LabelFilter(contains_any=building_level)
    asset_list = c.assets.list(asset_subtree_external_ids=str(AAA_asset), labels=my_label_filter, limit=-1)
    return asset_list

def find_TFM_AAA(c, root_asset):
    asset_list_AAA = c.assets.list(parent_external_ids=[str(root_asset)])
    return asset_list_AAA



In [3]:
root_assets_list_df = pd.DataFrame()

In [4]:
rbtn_cdf_login = widgets.RadioButtons(options=['energima', 'energima-dev'])
btn_cdf_login = widgets.Button(description='Login')
chkbx_cdf_login_show_status = widgets.Checkbox(description='Show status')

output_cdf_login = widgets.Output()


def cdf_login(p):
    
    global c, root_assets_list_df, building_list
    
    if rbtn_cdf_login.value == 'energima':
        api_key = 'M2ExY2U1ZmItNGRkNC00ZjVlLTgwMzctZDdjN2JhZDUzMjdj'
        c = CogniteClient(api_key=api_key, project="energima", client_name="energima")
    else:
        api_key_dev='ZmQ1NzIzNTktMmUzYi00MjA1LTg5OTYtMzc2NmNjNzZjZGRk'
        c = CogniteClient(api_key=api_key_dev, project="energima-dev", client_name="energima")
    
    rbtn_cdf_login.disabled=True
    btn_cdf_login.disabled = True
    
    #initial list of buildings
    root_assets_list_df = c.assets.list(root=True, limit=-1).to_pandas()
    building_list = df_columns_inst_to_list_of_tuples(root_assets_list_df, col_name='externalId')
    inputdropdown_ah_building_index.options = building_list
    update_building_drop_list.options = building_list
    update_building_update_meta_drop_list.options = building_list
    edit_building_drop_list.options = building_list
    
    
    with output_cdf_login:
        output_cdf_login.clear_output()
        if chkbx_cdf_login_show_status.value == True:
            display(print(c.login.status()))
        else:
            pass

btn_cdf_login.on_click(cdf_login)
        
widgets.VBox([widgets.HBox([rbtn_cdf_login, btn_cdf_login, chkbx_cdf_login_show_status]), output_cdf_login])

***

# Create new physical building

In [5]:
#gui

building_list=[]

lbl_building_selection = widgets.Label(value='Choose building:')

inputdropdown_ah_building_index = widgets.Dropdown(
    options=building_list,
    #value=building_list[0],
    description='Building:',
    layout={'width': '400px'}
)


AAA_list = []

input_rbtn_AAA = widgets.RadioButtons(
    options=AAA_list,
    layout={'width': '250pt'}, # If the items' names are long
    style={'description_width': '150px'},
    description='Choose sub_building:',
    disabled=False)

input_floors = widgets.BoundedIntText(
    value=1,
    min=0,
    max=999,
    description='Floors: ',
    disabled=True)


input_sub_floors = widgets.BoundedIntText(
    value=0,
    min=0,
    max=99,
    description='Sub Floors: ',
    disabled=True)

button_ah_building_index = widgets.Button(description="Choose Building")
output_ah_building_index = widgets.Output()

btn_select_sub_building = widgets.Button(description='Choose sub-building')
output_select_sub_building = widgets.Output()

btn_create_floors = widgets.Button(description='Create floors', icon='align-justify', disabled=True)
output_btn_create_floors = widgets.Output()

btn_create_all_rooms = widgets.Button(description='Create room structure', icon='clone', layout={'width':'200px'})
output_btn_create_all_rooms = widgets.Output()

btn_create = widgets.Button(description = 'Create building structure', icon='building', layout={'width':'200px'})
output_btn_create = widgets.Output()

In [6]:
#prog logics


def on_button_clicked_ah_building(b):

    global active_root_asset_id, active_root_asset, active_root_asset_dataSetId
    
    phys_building = False
    active_root_asset_id = inputdropdown_ah_building_index.value
    active_root_asset = root_assets_list_df.loc[active_root_asset_id].externalId
    active_root_asset_dataSetId = root_assets_list_df.loc[active_root_asset_id].dataSetId
    
    if active_root_asset != '':
        #update rbtn with sub_building info from active_root
        AAA_df = find_TFM_AAA(c=c, root_asset=active_root_asset).to_pandas()
        AAA_list = list(AAA_df.name)
        
        try:
            AAA_list.remove('Common')
            AAA_list.remove('TMP')
            
        except:
            pass
            
        input_rbtn_AAA.options = AAA_list
        
        
        input_floors.disabled = False
        input_sub_floors.disabled = False
        btn_create_floors.disabled = True
        
        btn_select_sub_building.disabled = False
        input_rbtn_AAA.disabled = False

        #shows radiobtn to choose between available AAA(A) and to 
        output = widgets.VBox([input_rbtn_AAA, btn_select_sub_building,output_select_sub_building, widgets.HBox([input_floors, input_sub_floors,
                                                                          btn_create_floors])])
        
    else:

        output = display(print('No building selected, no structure to show.'))
    
    #---
        
    with output_ah_building_index:
        output_ah_building_index.clear_output()
        
        output_btn_create_floors.clear_output()
        output_btn_create_all_rooms.clear_output()
        output_btn_create.clear_output()
        output_select_sub_building.clear_output()
        
        print('Building: ' + str(active_root_asset))

        display(output)

        
def select_sub_building_create(e):
    
    global my_target_building, my_target_subbuildings, my_target_floors, my_target_all_building_items
                
    #all these will be empty if no physical building exists
    my_target_buildings = find_item_on_level_in_phys_building_AAA(c=c, AAA_asset=str(active_root_asset + '+' + input_rbtn_AAA.value), building_level = ['building'])
    my_target_subbuildings = find_item_on_level_in_phys_building_AAA(c=c, AAA_asset=str(active_root_asset + '+' + input_rbtn_AAA.value), building_level = ['sub_building'])
    my_target_floors = find_item_on_level_in_phys_building_AAA(c=c, AAA_asset=str(active_root_asset + '+' + input_rbtn_AAA.value), building_level = ['floor'])
    my_target_rooms = find_item_on_level_in_phys_building_AAA(c=c, AAA_asset=str(active_root_asset + '+' + input_rbtn_AAA.value), building_level = ['room'])
    my_target_all_building_items = find_item_on_level_in_phys_building_AAA(c=c, AAA_asset=str(active_root_asset + '+' + input_rbtn_AAA.value), building_level=['building','sub_building','floor','room'])


    if len(my_target_all_building_items) > 0:

        #my_target_all_building_items = my_target_all_building_items.to_pandas()
        output = 'Physical building Found. Please go to section for "ADD Rooms" to edit the physical building structure.'

        input_floors.disabled = False
        input_sub_floors.disabled = False
        btn_create_floors.disabled = True
        
        btn_select_sub_building.disabled = False
        input_rbtn_AAA.disabled = False

    else:

        output = 'No physical building found under chosen rootAsset. Generating new building structure...'

        input_floors.disabled = False
        input_sub_floors.disabled = False
        btn_create_floors.disabled = False
        
        btn_select_sub_building.disabled = True
        input_rbtn_AAA.disabled = True
    
    with output_select_sub_building:
        output_select_sub_building.clear_output()
        display(print(output))
        
    
btn_select_sub_building.on_click(select_sub_building_create)        

#-----

def create_IntText_widget(no, value=1, descr='Floors', run_no=False):
    widget_list = []
    if run_no==True:
        x=1
    else:
        x=''
    i = 0
    for i in range(int(no)):
        #print(item)
        
        input_sub_buildings_floor = widgets.BoundedIntText(
            value=value,
            min=0,
            max=99,
            description=str(descr) + ' ' + str(x),
            disabled=False,
            style={'description_width': '150px'},
            layout={'width': '250px'}
        )
    
        widget_list.append(input_sub_buildings_floor)
        
        if run_no==True:
            x+=1
        
        i +=1
        
    return widget_list

def generate_room_widgets_from_floors_sub_floors(input_floors_val, input_sub_floors_val):
    """
    returns a tuple with 2x widget.VBox (floors , sub_floors). Each containing all labels_widgets and txt_widgets for all floors and underlying rooms
    """
    
    global floor_widget_list, sub_floor_widget_list, floors_all_widgets, sub_floors_all_widgets
    
    lbl_insert_number_of_rooms_in_floor = widgets.Label(value='Insert the number of rooms in each Floor (above celler)', layout={'width':'400px'})
    lbl_insert_number_of_subrooms_in_floor = widgets.Label(value='Insert the number of rooms in each Sub Floor (celler)')
    
    floor_widget_list = create_IntText_widget(no=input_floors_val, value=1, descr='Rooms in Floor#', run_no=True)
    sub_floor_widget_list = create_IntText_widget(no=input_sub_floors_val, value=1, descr= 'Rooms in Sub-Floor#', run_no=True)
    
    floors_all_widgets = widgets.VBox([x for x in floor_widget_list])
    sub_floors_all_widgets = widgets.VBox([y for y in sub_floor_widget_list])
    
    floor_lbl_txt_widget_box = widgets.VBox([lbl_insert_number_of_rooms_in_floor,floors_all_widgets])
    subfloor_lbl_txt_widget_box = widgets.VBox([lbl_insert_number_of_subrooms_in_floor, sub_floors_all_widgets])
    
    return floor_lbl_txt_widget_box, subfloor_lbl_txt_widget_box
    

#GUI func

def generate_sub_building_widget(b):        
    
    global floor_box, subfloor_box

    floor_box, subfloor_box = generate_room_widgets_from_floors_sub_floors(input_floors.value, input_sub_floors.value)
    
    with output_btn_create_floors:
        output_btn_create_floors.clear_output()
        display(widgets.VBox([widgets.HBox([floor_box, subfloor_box]), btn_create_all_rooms]))
        

        
        
        
#functions

def create_tab_with_rooms_pr_floor(floors_widget_list, sub=False):
    titles = []
    list_floor_widgets  = []
    for floor in floors_widget_list:
        list_room_widgets  = []
        
        if sub == False:
            level_name = 'Floor'
            flor_level_start_no = 1
        else:
            level_name = 'Sub-Floor'
            flor_level_start_no = 1
            
        titles.append(((int(floors_widget_list.index(floor))), str(level_name) + ' ' + str(int(floors_widget_list.index(floor) + flor_level_start_no))))

        for room in floor:
            #print('Room: '  + str(floor.index(room)))
            dipl_list=[]
            for item in room:
                #display(item)
                dipl_list.append(item)
            list_room_widgets.append(widgets.HBox(dipl_list))
        list_floor_widgets.append(widgets.VBox(list_room_widgets))

    building_tabs = widgets.Tab(children=list_floor_widgets)

    for title in titles:
        building_tabs.set_title(title[0],title[1])

    return building_tabs

#---


def create_room_widgets(floor_widget_list, sub=False):
    #subfloors_list = []
    floors_widget_list = []

    if sub == True:
        zfill=2
        floor_prefix = 'K'
        floor_no_start=1
    else:
        zfill=3
        floor_prefix = ''
        floor_no_start=1
    
    floor_no=floor_no_start
    
    for floor_widget in floor_widget_list:
        #floor_no=x #floor_widget_list.index(floor_widget)
        #print(item.description)
        #print(item.value)
        #print(x)
        
        room_no=0
        rooms_widget_list=[]
        for room_no in range(floor_widget.value):
            #room_no=i

            extId = str(active_root_asset) + '+' + str(input_rbtn_AAA.value) + '=200.' + str(floor_prefix) + str(floor_no).zfill(int(zfill)) + '-FR' + str(room_no).zfill(3)

            label_extId_widget = widgets.Label(value=extId, layout={'width': '350px'})

            input_name_widget = widgets.Text(
                value='',
                placeholder='F' + str(floor_no) + ' R' + str(room_no) + ' (name)',
                description= 'Name:',
                disabled=False)

            input_descr_widget = widgets.Text(
                value='',
                placeholder='F' + str(floor_no) + ' R' + str(room_no) + ' (description)',
                description= 'Descr.:',
                disabled=False)

            rooms_widget_list.append([label_extId_widget, input_name_widget, input_descr_widget])
        floors_widget_list.append(rooms_widget_list)
        floor_no +=1

    #------
    return floors_widget_list


def generate_building_widgets_with_tabs_pr_floor(b):
    
    global active_widgets, active_sub_widgets, building_tabs, sub_building_tabs
    
    active_widgets = create_room_widgets(floor_widget_list)
    building_tabs = create_tab_with_rooms_pr_floor(active_widgets)
    
    active_sub_widgets = create_room_widgets(sub_floor_widget_list, sub=True)
    sub_building_tabs = create_tab_with_rooms_pr_floor(active_sub_widgets, sub=True)
    
    with output_btn_create_all_rooms:
        output_btn_create_all_rooms.clear_output()
        
        display(widgets.VBox([building_tabs, sub_building_tabs, widgets.HBox([btn_create])]))
        
# btn_create_all_rooms.on_click(generate_building_widgets_with_tabs_pr_floor)
# widgets.HBox([output_btn_create_all_rooms])

def create_building_structure_dict(floors_widget_list):
    room_dict_list = []
    for floor in floors_widget_list:
        for room in floor:
            widg_value_list = []
            for widg in room:
                widg_value_list.append(widg.value)

            extId = widg_value_list[0]
            room_name = widg_value_list[1]
            floor_no = extId[extId.find('.')+1 : extId.find('-FR')]
            room_no = extId[-3:]
            desription = widg_value_list[2]

            room_dict = {'extId': extId,
            'AAA': input_rbtn_AAA.value,
            'floor_no': floor_no,
            'room_no': room_no,
            'room_name': widg_value_list[1],
             'description': desription #,'type':type_item
            }
            room_dict_list.append(room_dict)
    return room_dict_list


def generate_total_building_structure_dict(b):

    global building_structure_dict, sub_building_structure_dict, EEP_building_client_create
    
    EEP_building_client_create = PhysicalBuildingAPI(cdf_project=str(rbtn_cdf_login.value), root_external_id=active_root_asset, dryrun=False)
    
    building_structure_dict = create_building_structure_dict(active_widgets)
    sub_building_structure_dict = create_building_structure_dict(active_sub_widgets)
    
    for room in building_structure_dict:
        EEP_building_client_create.add_room(AAA = room['AAA'], floor = room['floor_no'], name = room['room_name'], description = room['description'])        
    
    for sub_room in sub_building_structure_dict:
        EEP_building_client_create.add_room(AAA = sub_room['AAA'], floor = sub_room['floor_no'], name = sub_room['room_name'], description = sub_room['description'])        
    
    btn_upload_building_structure_to_eep.disabled=False
    
    
    with output_btn_create:
        output_btn_create.clear_output()
        EEP_building_client_create.dump_building_structure(raw=False)

#-----
            
active_root_asset = ''

btn_upload_building_structure_to_eep = widgets.Button(description='UPLOAD NEW BUILDING TO EEP', disabled=True, layout = {'width' : '200pt'})
btn_upload_building_structure_to_eep.style.button_color = 'lightgreen'
                              
#building_list = df_columns_inst_to_list_of_tuples(root_assets_list_df, col_name='externalId')
building_list = []
inputdropdown_ah_building_index.options = building_list

#gui-action
button_ah_building_index.on_click(on_button_clicked_ah_building)

#gui-layout
header_box = widgets.HBox([lbl_building_selection])
top_box = widgets.HBox([inputdropdown_ah_building_index, button_ah_building_index])
bottom_box = widgets.HBox([output_ah_building_index])
widgets.VBox([header_box, top_box, bottom_box])

# --- END choosing building and checking for sub buildings in selected building

In [7]:
btn_create_floors.on_click(generate_sub_building_widget)
widgets.HBox([output_btn_create_floors])

In [8]:
btn_create_all_rooms.on_click(generate_building_widgets_with_tabs_pr_floor)
widgets.HBox([output_btn_create_all_rooms])

In [9]:
btn_create.on_click(generate_total_building_structure_dict)
widgets.HBox([output_btn_create])

In [10]:
output_upload_building_to_eep = widgets.Output()

f0_max_count = 2

f0 = widgets.IntProgress(min=0, max=f0_max_count) # instantiate the bar

def upload_building_to_eep(m):
    f0.value = 0
    f0.value += 1
    
    EEP_building_client_create.commit()
    f0.value += 1
    
    with output_upload_building_to_eep:
        output_upload_building_to_eep.clear_output()
        print('Uploaded to EEP!')
        
        EEP_building_client_create.dump_building_structure(raw=False)

    
btn_upload_building_structure_to_eep.on_click(upload_building_to_eep)

widgets.VBox([widgets.HBox([btn_upload_building_structure_to_eep, f0]), output_upload_building_to_eep])

***

# Update physical building

## Add rooms

In [11]:
#root_assets_list_df = c.assets.list(root=True, limit=-1).to_pandas()
# building_ser = root_assets_list_df.externalId
# building_list= building_ser.tolist()
# building_list.sort()
building_list = []

update_building_drop_list = widgets.Dropdown(
    options=building_list,
    #value=building_list[0],
    description='Building:',
    layout={'width': '400px'}
)

btn_choose_building_update = widgets.Button(description="Choose Building", icon='city', layout={'width': '200px'})
output_choose_building_update = widgets.Output()

widgets.HBox([update_building_drop_list, btn_choose_building_update])

In [12]:
#active_update_root_asset

In [13]:
def choose_building_and_create_sub_building_rbtn(a):
    
    global active_update_root_asset, floor_names_list, input_rbtn_update_AAA, update_rooms_widgets_list, rooms_widget_list
    
    active_update_root_asset = root_assets_list_df.loc[update_building_drop_list.value].externalId
    
    
    AAA_asset_update_df = c.assets.list(parent_external_ids=[str(active_update_root_asset)]).to_pandas()
    
    floor_names_list = find_number_of_floors_in_active_building(active_update_root_asset)
    
    if len(floor_names_list) > 0:
    
        if len(AAA_asset_update_df) > 0:

            AAA_update_list = list(AAA_asset_update_df.name)

            try:
                AAA_update_list.remove('Common')
            except:
                pass

            input_rbtn_update_AAA = widgets.RadioButtons(
                options=AAA_update_list,
                layout={'width': '250pt'}, # If the items' names are long
                style={'description_width': '150px'},
                description='Choose sub_building:',
                disabled=False)

            #input_rbtn_update_AAA.options = AAA_update_list
            input_rbtn_update_AAA.disabled = False
            btn_choose_sub_building_update.disabled = False
            update_rooms_widgets_list = []
            rooms_widget_list = []
            

            output = widgets.HBox([input_rbtn_update_AAA, btn_choose_sub_building_update])
        else:

            output = str('No AAA found')
            
    else:
        output = str('NO physical building found to update. Please create physical building.')

    with output_choose_building_update:
        output_choose_building_update.clear_output()
        output_choose_sub_building_update.clear_output()
        add_rooms_update_content_output.clear_output()
        building_update_structure_content_output.clear_output()
        output_upload_add_rooms_to_eep.clear_output()
        
        
        #output = str(active_update_root_asset)
        display(output)

btn_choose_sub_building_update = widgets.Button(description="Choose Sub-Building", layout={'width': '200px'})
output_choose_sub_building_update = widgets.Output()

btn_choose_building_update.on_click(choose_building_and_create_sub_building_rbtn)
#widgets.HBox([output_choose_building_update, btn_choose_sub_building_update])
widgets.HBox([output_choose_building_update])

In [14]:
btn_add_room_update = widgets.Button(description="Add room", icon='plus')
btn_subtract_room_update = widgets.Button(description="Remove room", icon='minus')
btn_add_floor_name_update = widgets.Button(description="Add floor")
lbl_spacer = widgets.Label(value="", layout={'width': '200pt'})

lbl_new_floor_in_list = widgets.Label(value="If floor number not in list:")

inttext_floor_number = widgets.BoundedIntText(
    value=1,
    min=0,
    max=999,
    step=1,
    description='New Floor: ',
    disabled=False
)

rbtn_floor_sub_floor = widgets.RadioButtons(
    options=['Floor', 'Sub_Floor'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='',
    disabled=False
)


update_rooms_widgets_list = []

def choose_sub_building_and_create_add_remove_buttons(b):
    
    #global inttext_floor_number
    
    input_rbtn_update_AAA.disabled = True
    btn_choose_sub_building_update.disabled = True
    update_rooms_widgets_list = []
    
    
    output = widgets.HBox([btn_add_room_update, btn_subtract_room_update, lbl_spacer, widgets.VBox([lbl_new_floor_in_list, inttext_floor_number,rbtn_floor_sub_floor,btn_add_floor_name_update])])
    
    with output_choose_sub_building_update:
        output_choose_sub_building_update.clear_output()
        display(output)
        
btn_choose_sub_building_update.on_click(choose_sub_building_and_create_add_remove_buttons)

widgets.HBox([output_choose_sub_building_update])

In [15]:
def add_floor_to_floor_list(d):
    
    floor_names_in_AAA_list = find_number_of_floors_in_active_building(active_update_root_asset + '+' + str(input_rbtn_update_AAA.value))
    
    new_floor_no = inttext_floor_number.value
    
    if rbtn_floor_sub_floor.value == 'Floor':
        new_floor_no_str = str(new_floor_no).zfill(3)
    else:
        new_floor_no_str = str('K') + str(new_floor_no).zfill(2)
    
    with output_choose_sub_building_update:
        if new_floor_no_str not in floor_names_in_AAA_list:
            floor_names_in_AAA_list.append(new_floor_no_str)
            floor_names_in_AAA_list.sort()
            #input_floor_select_widget.option = floor_names_in_AAA_list
            
            #update existing widgets
            for item in update_rooms_widgets_list:
                item[2].options = floor_names_in_AAA_list
            
        else:
            print(str(new_floor_no) + ' already in list of floors.')
            
        #print(str(new_floor_no_str) + '  ' + str(floor_names_in_AAA_list))

btn_add_floor_name_update.on_click(add_floor_to_floor_list)

In [16]:
add_rooms_update_content_output = widgets.Output()

btn_create_updated_building_structure_dict = widgets.Button(description='Create updated building structure', layout={'width' : '400pt'})

def add_room_to_update_rooms_widgets_list(c):
    
    global rooms_widget_list, update_rooms_widgets_list, input_floor_select_widget
    
    floor_names_in_AAA_list = find_number_of_floors_in_active_building(active_update_root_asset + '+' + str(input_rbtn_update_AAA.value))
    
    input_name_widget = widgets.Text(
        value='',
        placeholder='(room name)',
        description= 'Room name:',
        disabled=False)

    input_descr_widget = widgets.Text(
        value='',
        placeholder='(description)',
        description= 'Description:',
        disabled=False)
    
    input_floor_select_widget = widgets.Dropdown(
        options=floor_names_in_AAA_list,
        value=floor_names_in_AAA_list[0],
        description='Floor #:',
        disabled=False
    )

    rooms_widget_list = [input_name_widget, input_descr_widget, input_floor_select_widget]
    #output = widgets.VBox([x for x in rooms_widget_list])
    update_rooms_widgets_list.append(rooms_widget_list)
    #input_floor_select_widget
    #update_rooms_widgets_list
    #rooms_widget_list.append([input_name_widget, input_descr_widget, input_floor_select_widget])
    
    with add_rooms_update_content_output:
        add_rooms_update_content_output.clear_output()
        for item in update_rooms_widgets_list:
            display(widgets.HBox([x for x in item]))
            
        display(btn_create_updated_building_structure_dict)

btn_add_room_update.on_click(add_room_to_update_rooms_widgets_list)
widgets.HBox([add_rooms_update_content_output])

In [17]:
update_rooms_widgets_list
#rooms_widget_list

[]

In [18]:
def find_number_of_floors_in_active_building(active_update_root_asset):
    my_label_filter = LabelFilter(contains_any=["floor", "sub_floor"])
    floors_asset_df = c.assets.list(asset_subtree_external_ids=active_update_root_asset, labels=my_label_filter, limit=-1).to_pandas()
    
    foor_number_list = []
    
    if len(floors_asset_df) > 0:
    
        floors_asset_list = floors_asset_df.name.tolist()

        for foor_name in floors_asset_list:
            active_floor_name = foor_name.split('.',1)
            foor_number_list.append(active_floor_name[1])

        foor_number_list.sort()
    
    
    return foor_number_list

In [19]:
def subtract_room_to_update_rooms_widgets_list(c):
    
    #global rooms_widget_list
    last_index = len(update_rooms_widgets_list) -1
    #print(last_index)
    if last_index >= 0:
    
        update_rooms_widgets_list.pop(last_index)
        
    else:
        pass
    
    with add_rooms_update_content_output:
        add_rooms_update_content_output.clear_output()
        for item in update_rooms_widgets_list:
            display(widgets.HBox([x for x in item]))
        
        display(btn_create_updated_building_structure_dict)
        
btn_subtract_room_update.on_click(subtract_room_to_update_rooms_widgets_list)

In [20]:
def create_room_update_widgets():
    
    #label_extId_widget = widgets.Label(value=extId, layout={'width': '350px'})

    input_name_widget = widgets.Text(
        value='',
        placeholder='(room name)',
        description= 'Room name:',
        disabled=False)

    input_descr_widget = widgets.Text(
        value='',
        placeholder='(description)',
        description= 'Description:',
        disabled=False)
    
    input_floor_select_widget = widgets.Dropdown(
        options=['1', '2', '3'],
        value='2',
        description='Floor #:',
        disabled=False
    )

    rooms_widget_list = [input_name_widget, input_descr_widget, input_floor_select_widget]

    return rooms_widget_list

In [21]:
add_room_dict_list = []

def create_updated_building_structure_dict(f):
    
    global add_room_dict_list, EEP_building_client_add, update_rooms_widgets_list
    
    add_room_dict_list = []
    
    for add_room in update_rooms_widgets_list:
        add_widg_value_list = []

        for add_widg in add_room:
            
            add_widg_value_list.append(add_widg.value)


        add_room_dict = {'AAA': input_rbtn_update_AAA.value,
            'floor_no': add_widg_value_list[2],
            'room_name': add_widg_value_list[0],
            'description': add_widg_value_list[1] #,'type':type_item
        }
        add_room_dict_list.append(add_room_dict)
    
    #print debug
    print(add_room_dict_list)
    
    #creates instance of eep-api
    EEP_building_client_add = PhysicalBuildingAPI(cdf_project=str(rbtn_cdf_login.value), root_external_id=active_update_root_asset, dryrun=False)
    
    for item in add_room_dict_list:    
        EEP_building_client_add.add_room(AAA = item['AAA'],
                                         name = item['room_name'],
                                         floor = item['floor_no'],
                                         description = item['description'])
    
    btn_upload_add_rooms_to_eep.disabled = False
    
    with building_update_structure_content_output:
        building_update_structure_content_output.clear_output()
        EEP_building_client_add.dump_building_structure()
        #display(add_room_dict_list)
    
building_update_structure_content_output = widgets.Output()

btn_create_updated_building_structure_dict.on_click(create_updated_building_structure_dict)
widgets.HBox([building_update_structure_content_output])

In [22]:
btn_upload_add_rooms_to_eep = widgets.Button(description='UPLOAD ADDED ROOMS TO EEP', disabled=True, layout = {'width' : '200pt'})
btn_upload_add_rooms_to_eep.style.button_color = 'lightgreen'
output_upload_add_rooms_to_eep = widgets.Output()

f1_max_count = 2

f1 = widgets.IntProgress(min=0, max=f1_max_count) # instantiate the bar

def upload_add_rooms_to_eep(t):
    f1.value = 0
    f1.value +=1
    
    EEP_building_client_add.commit()
    f1.value +=1
    
    with output_upload_add_rooms_to_eep:
        output_upload_add_rooms_to_eep.clear_output()
        
        EEP_building_client_add.dump_building_structure()

        
btn_upload_add_rooms_to_eep.on_click(upload_add_rooms_to_eep)

widgets.VBox([widgets.HBox([btn_upload_add_rooms_to_eep, f1]), output_upload_add_rooms_to_eep])

***

## Edit rooms (names & descriptions)

In [23]:
building_list = []

edit_building_drop_list = widgets.Dropdown(
    options=building_list,
    #value=building_list[0],
    description='Building:',
    layout={'width': '400px'}
)

edit_show_dump = widgets.Checkbox(
    value=False,
    description='Show building structure dump',
    disabled=False,
    layout = {'width': '300px'}
)

btn_choose_building_edit = widgets.Button(description="Choose Building", icon='city', layout={'width': '200px'})
output_choose_building_edit = widgets.Output()

widgets.VBox([widgets.HBox([edit_building_drop_list, btn_choose_building_edit]), output_choose_building_edit])


In [24]:
def choose_building_and_create_sub_building_rbtn_edit(a):
    
    global active_edit_root_asset, floor_names_list, input_rbtn_edit_AAA, edit_show_dump
    
    active_edit_root_asset = root_assets_list_df.loc[edit_building_drop_list.value].externalId
    
    
    AAA_asset_edit_df = c.assets.list(parent_external_ids=[str(active_edit_root_asset)]).to_pandas()
    
    floor_names_list = find_number_of_floors_in_active_building(active_edit_root_asset)
    
    if len(floor_names_list) > 0:
    
        if len(AAA_asset_edit_df) > 0:

            AAA_edit_list = list(AAA_asset_edit_df.name)

            try:
                AAA_edit_list.remove('Common')
            except:
                pass

            input_rbtn_edit_AAA = widgets.RadioButtons(
                options=AAA_edit_list,
                layout={'width': '250pt'}, # If the items' names are long
                style={'description_width': '150px'},
                description='Choose sub_building:',
                disabled=False)

            
            output = widgets.HBox([input_rbtn_edit_AAA, btn_choose_sub_building_edit, edit_show_dump])
            
            btn_choose_sub_building_edit.disabled=False
            input_rbtn_edit_AAA.disabled=False
            btn_update_building_structure_edit.disabled = True
            btn_upload_edit_building_structure_to_eep.disabled = True
            
            
        else:

            output = str('No AAA found')
            
    else:
        output = str('NO physical building found to edit. Please create physical building.')

    with output_choose_building_edit:
        output_choose_building_edit.clear_output()
        output_choose_sub_building_edit.clear_output()
        output_update_building_structure_edit.clear_output()
        output_upload_edit_building_to_eep.clear_output()
               
        display(output)

btn_choose_sub_building_edit = widgets.Button(description="Choose Sub-Building", disabled=True, layout={'width': '200px'})
output_choose_sub_building_edit = widgets.Output()

btn_choose_building_edit.on_click(choose_building_and_create_sub_building_rbtn_edit)
#widgets.HBox([output_choose_sub_building_edit])

In [25]:
#sub_building_structure

In [26]:
# for floor_no, rooms_dict in sub_building_structure.items():
#     if floor_no != 'metadata':
#         print('---')
#         print('Etg: ' + str(floor_no))
# #        print(type(rooms_dict))
#         for k, v in rooms_dict.items():
#             if k != 'metadata':
#                 print(k, v)
#                 print(type(v))
#                 print(v.get('name'))

In [27]:

btn_update_building_structure_edit = widgets.Button(description='Update building structure', disabled=True, layout = {'width':'250px'})
output_update_building_structure_edit = widgets.Output()

def btn_choose_sub_building_edit_click(h):
    
    global EEP_building_client_edit, edit_widg_list, sub_building_structure
    
    EEP_building_client_edit = PhysicalBuildingAPI(cdf_project=str(rbtn_cdf_login.value),
                                                   root_external_id=active_edit_root_asset,
                                                   dryrun=False)
    
    building_structure_edit_dict = EEP_building_client_edit.list()
    sub_building_structure = building_structure_edit_dict.get(str(input_rbtn_edit_AAA.value))
    
    edit_widg_list = []

    for floor_no, rooms_dict in sub_building_structure.items():
        #print(floor_no)
        if floor_no != 'metadata':
        
            edit_widg_room_list = []
            for room_id, room_content in rooms_dict.items():

                if room_id != 'metadata':
                    print('room_content: ' + str(room_content))
                    print('room_content type: ' + str(type(room_content)))
                    active_edit_widg_room_list = [widgets.Label(value='Floor: '),
                                                  widgets.Label(value=str(floor_no)),
                                                  widgets.Label(value='Room_ID: '),
                                                  widgets.Label(value=str(room_id)),
                                       widgets.Text(value=room_content.get('name'), description='name:'),
                                       widgets.Text(value=room_content.get('desc'), description='desc:')]

                    edit_widg_room_list.append(active_edit_widg_room_list)

            edit_widg_list.append(edit_widg_room_list)
    
    
    btn_update_building_structure_edit.disabled = False
    btn_choose_sub_building_edit.disabled=True
    input_rbtn_edit_AAA.disabled=True
    
    with output_choose_sub_building_edit:
        output_choose_sub_building_edit.clear_output()
        
        if edit_show_dump.value == True:
            display(EEP_building_client_edit.dump_building_structure())
        
        for widg_floor in edit_widg_list:
            for widg in widg_floor:
                display(widgets.HBox(widg))
        
        
    
btn_choose_sub_building_edit.on_click(btn_choose_sub_building_edit_click)
widgets.VBox([output_choose_sub_building_edit, btn_update_building_structure_edit])

In [28]:
def edit_building_structure_dict(floors_widget_list):
    room_dict_list = []
    for floor in floors_widget_list:
        for room in floor:
            widg_value_list = []
            for widg in room:
                widg_value_list.append(widg.value)

            room_name = widg_value_list[4]
            floor_no = widg_value_list[1]
            room_id = widg_value_list[3]
            desription = widg_value_list[5]

            room_dict = {'AAA': input_rbtn_edit_AAA.value,
                        'floor_no': floor_no,
                        'room_id': room_id,
                        'room_name': room_name,
                         'description': desription #,'type':type_item
                    }
            room_dict_list.append(room_dict)
    return room_dict_list


def update_building_structure_edit(l):
    
    new_building_structure_dict = edit_building_structure_dict(edit_widg_list)
    
    for room_item in new_building_structure_dict:

        room_id = room_item.get('room_id')
        AAA = room_item.get('AAA')
        floor = room_item.get('floor_no')
        name = room_item.get('room_name')
        description = room_item.get('description')
        
        EEP_building_client_edit.update_room(room_id=room_id,
                                             AAA=AAA,
                                             floor=floor,
                                             name=name,
                                             description= description)
        
        btn_upload_edit_building_structure_to_eep.disabled = False
        
    with output_update_building_structure_edit:
        output_update_building_structure_edit.clear_output()
        
        display(EEP_building_client_edit.dump_building_structure())
    
btn_update_building_structure_edit.on_click(update_building_structure_edit)
widgets.VBox([output_update_building_structure_edit])

In [29]:
btn_upload_edit_building_structure_to_eep = widgets.Button(description='UPLOAD EDITED ROOMS TO EEP', disabled=True, layout = {'width' : '200pt'})
btn_upload_edit_building_structure_to_eep.style.button_color = 'lightgreen'
output_upload_edit_building_to_eep = widgets.Output()

f_edit_max_count = 2

f_edit = widgets.IntProgress(min=0, max=f_edit_max_count) # instantiate the bar

def upload_edit_building_to_eep(o):
    f_edit.value = 0
    f_edit.value += 1
    
    EEP_building_client_edit.commit()
    f_edit.value += 1
    
    with output_upload_edit_building_to_eep:
        output_upload_edit_building_to_eep.clear_output()
        print('Uploaded to EEP!')
        
        EEP_building_client_edit.dump_building_structure(raw=False)

    
btn_upload_edit_building_structure_to_eep.on_click(upload_edit_building_to_eep)

widgets.VBox([widgets.HBox([btn_upload_edit_building_structure_to_eep, f_edit]), output_upload_edit_building_to_eep])

***

## Delete rooms in existing building

In [30]:
#building_structure.delete(room_id="009", AAA="GF", floor="001")

Functionality not yet released. Contact EEP-Backend for manual delete.

***

## Update building with metadata

In [31]:
def get_room_type_list_from_raw():
    room_types_df = c.raw.rows.retrieve("energima_sensormaps", "tfm", "room").columns
    room_types_list = []
    for r, k in room_types_df.items():
        if isinstance(k, dict):
            #print(k.get("name"))
            room_types_list.append(k.get("name"))
    
    room_types_list.sort()
    
    return room_types_list

def get_all_rooms_from_eep_api():
    building_structure_list = building_structure_client.list()
    
    
    return
    

def create_building_widgets(building_cat_list, building_type_list):
    
    global building_type_widget_list
    
    building_type_widget_list = []
    for item in building_cat_list:
        building_type_widget_list.append(widgets.Select(
            options=building_type_list[int(building_cat_list.index(item))],
            disabled=False
        ))

    accordion = widgets.Accordion(children=building_type_widget_list)

    for item in building_cat_list:
        accordion.set_title(building_cat_list.index(item), str(item))
    
    return accordion


def prep_building_type_and_cat_list_info():
    building_type_list = []
    building_cat_list = []
    building_types_dict = c.raw.rows.retrieve("energima_sensormaps", "tfm", "building").columns
    for k, v in building_types_dict.items():
        if isinstance(v, dict):
            pass
        else:
            building_type_list.append(v)
            building_cat_list.append(k)
    return building_type_list, building_cat_list


def get_room_list_from_eep_api(AAA_val):
    building_structure_list = building_structure_client.list(AAA=AAA_val)
    metadata_list = []
    floor_list = []
    room_list = []
    
    for sub_build_k, sub_build_v in building_structure_list.items():
        for floor_k, floor_v in sub_build_v.items():
            if floor_k != 'metadata':
            
                for room_k, room_v in floor_v.items():
                    if room_k != 'metadata':
                        room_total_info_str = str('AAA: ' + str(sub_build_k) + ' -> Floor no: ' + str(floor_k) + ' -> Room no: ' + str(room_k) + '  ' + 'Room content: ' + str(room_v))
                        room_list.append(room_total_info_str)
                
    return room_list

def get_values_from_selected_room_list(select_room_list_values):
    """
    values output from widget -> select_room_list.value (tuple)
    """
    
    data_as_string_list = list(select_room_list_values)
    res_list = []
    for item in data_as_string_list:
        active_data_item_list = []
        active_data_item_list = item.split(' ')
        if len(active_data_item_list) > 10:
            res_list.append([active_data_item_list[1], active_data_item_list[5], active_data_item_list[9]])
        else:
            print('Error extracting data from: ' + str(item))
        #remove if successful
    return res_list

In [32]:

building_update_meta_list = []

update_building_update_meta_drop_list = widgets.Dropdown(
    options=building_list,
    #value=building_update_meta_list[0],
    description='Building:',
    layout={'width': '400px'}
)


chkbx_update_building_update_meta_show_building_structure = widgets.Checkbox(
    value=True,
    description='Show building structure',
    disabled=False
)

btn_choose_building_update_meta = widgets.Button(description="Choose Building", layout={'width': '300px'})

widgets.HBox([update_building_update_meta_drop_list, btn_choose_building_update_meta, chkbx_update_building_update_meta_show_building_structure])

In [33]:
output_choose_building_update_meta = widgets.Output()

btn_choose_metadata_to_update = widgets.Button(description='Select', layout={'width':'250pt'}, disabled=True)

rbtn_choose_metadata_to_update = widgets.RadioButtons(
    options=['Building/floor category', 'Area', 'Room Type'],
    value='Room Type',
    description='Choose metadata type:',
    disabled=True,
    layout={'width':'350pt'},
    style= {'description_width': '200px'}
)

AAA_meta_update_list = []

input_rbtn_AAA_meta_update = widgets.RadioButtons(
    options=AAA_meta_update_list,
    layout={'width': '250pt'}, # If the items' names are long
    style={'description_width': '150px'},
    description='Choose sub_building:',
    disabled=False)

       
def choose_building_update_meta(g):
    
    global active_update_meta_root_asset, building_structure_client
    
    active_update_meta_root_asset = root_assets_list_df.loc[update_building_update_meta_drop_list.value].externalId
    #print(active_update_meta_root_asset)
    building_structure_client = PhysicalBuildingAPI(cdf_project="energima",
                                                    root_external_id=active_update_meta_root_asset,
                                                    dryrun=False)
    
    rbtn_choose_metadata_to_update.disabled = False
    btn_choose_metadata_to_update.disabled = False
    
    #--
    AAA_meta_update_df = find_TFM_AAA(c=c, root_asset=active_update_meta_root_asset).to_pandas()
    AAA_meta_update_list = list(AAA_meta_update_df.name)
    try:
        AAA_meta_update_list.remove('Common')
    except Exception as err:
        print(err)
        pass
    
    input_rbtn_AAA_meta_update.options = AAA_meta_update_list
    
    #enables button if already used and rerun
    btn_choose_building_update_meta.disabled = False
    rbtn_choose_metadata_to_update.disabled = False
    input_rbtn_AAA_meta_update.disabled = False
    
    #--
    
    with output_choose_building_update_meta:
        output_choose_building_update_meta.clear_output()
        output_choose_type_update_meta.clear_output()
        output_area_widgets.clear_output()
        output_add_type_to_room_and_remove_item_from_list.clear_output()
        output_upload_building_types_and_cat_to_eep.clear_output()
        output_uplad_all_types_to_eep.clear_output()
        
    
        if chkbx_update_building_update_meta_show_building_structure.value == True:

            building_structure_conent = building_structure_client.dump_building_structure(raw=False)
            
        else:
            display(str(active_update_meta_root_asset))

btn_choose_building_update_meta.on_click(choose_building_update_meta)


widgets.VBox([output_choose_building_update_meta, input_rbtn_AAA_meta_update, widgets.HBox([rbtn_choose_metadata_to_update, btn_choose_metadata_to_update])])

In [34]:
def read_floor_metadata(building_structure_client, floors_list):
    floors_metadata_list = []
    
    for floor in floors_list:
        
        floor_metadata = building_structure_client.list_metadata(AAA=input_rbtn_AAA_meta_update.value, floor=floor)
        
        if floor_metadata is not None:
        
            data = {
                'floor_no' : str(floor),
                'building_type' : floor_metadata.get('building_type', ''),
                'building_category' : floor_metadata.get('building_category', '')
            }
            
        else:
            
            data = {
                'floor_no' : str(floor),
                'building_type' : '',
                'building_category' : ''
            }
            
    
        floors_metadata_list.append(data)
    return floors_metadata_list


def update_floor_metadata(floors_metadata_list, floor_no, building_type, building_category):
    
    building_type_str = building_type

    building_category_str = building_category

    
    for item in floors_metadata_list:
        
        old_building_type_list = []
        old_building_cat_list = []
        
        if item['floor_no'] == floor_no:
            
            if len(item['building_type']) > 0:
                old_building_type_list = item['building_type'].split(";")
            else:
                old_building_type_list = []
            
            if len(item['building_category']) > 0:
                old_building_cat_list = item['building_category'].split(";")
            else:
                old_building_cat_list = []
            
            if building_type_str not in old_building_type_list:  
                old_building_type_list.append(building_type_str)
                item['building_type'] = ";".join(old_building_type_list)
                
            if building_category_str not in old_building_cat_list:
                old_building_cat_list.append(building_category_str)
                item['building_category'] = ";".join(old_building_cat_list)
                

def clear_floor_metadata(floors_metadata_list, floor_no):
    for item in floors_metadata_list:
        if item['floor_no'] == floor_no:
            item['building_type'] = ''
            item['building_category'] = ''


#----start gui

chk_commit_upload_building_types_and_cat_to_eep = widgets.Checkbox(description='Dry run', value=False)
output_choose_type_update_meta = widgets.Output()
btn_add_type_to_room = widgets.Button(description='Add type to rooms', layout={'width':'800px', 'height': '50px'})
lbl_spacer_update_metadata_type = widgets.Label(layout={'height':'30px'})
lbl_spacer_floor_reset = widgets.Label(value='NB! A type must be chosen for each floor. Unselected floors will get type & category None')

btn_add_types_to_floors = widgets.Button(description='Add')
btn_clear_types_to_floors = widgets.Button(description='Clear types from selected floors', layout = {'width':'300px'})
btn_clear_types_to_floors.style.button_color = 'yellow'

def choose_type_update_meta(i):
    global floor_metadata_dict_list, floors_type_metadata, floor_no_metaupdate_list, room_list, room_type_update_meta_list, AAA_meta_update_selected, select_roomtype_list, select_room_list, btn_add_type_to_room, data_list_for_room_type_update_type, data_list_for_room_type_update, floors_type_metadata, input_floors_metadata_multiselect, widget_building_type, building_type_list, building_cat_list, rbtn_choose_building_level_area
    
    AAA_meta_update_selected = input_rbtn_AAA_meta_update.value

    if rbtn_choose_metadata_to_update.value == 'Building/floor category':
        
        #collecting building cathegories and types from raw and creates accordion widgets
        building_type_list, building_cat_list = prep_building_type_and_cat_list_info()
        widget_building_type = create_building_widgets(building_cat_list, building_type_list)
        
       
        building_dict_temp = building_structure_client.list(AAA=AAA_meta_update_selected)
        floor_no_metaupdate_list = []
        for k, v in building_dict_temp.items():
            for k1, v1 in v.items():
                #print(str(k1))
                if k1 != 'metadata':
                    floor_no_metaupdate_list.append(k1)
                
        input_floors_metadata_multiselect = widgets.SelectMultiple(
            options=floor_no_metaupdate_list,
            description='Floors:',
            disabled=False,
            layout = {'width' : '400px', 'height' : '400px'}
        )
                
        floor_metadata_dict_list = read_floor_metadata(building_structure_client=building_structure_client,
                                                       floors_list=floor_no_metaupdate_list)
        
        floors_type_metadata = widgets.SelectMultiple(
            options=floor_metadata_dict_list,
            description='Floors metadata:',
            disabled=True,
            layout = {'width' : '400px', 'height' : '400px'}
        )
        
        
        button_box = widgets.HBox([btn_add_types_to_floors, btn_clear_types_to_floors])
        
        content_box = widgets.HBox([widget_building_type, widgets.VBox([lbl_spacer_floor_reset, input_floors_metadata_multiselect, button_box])])
        
        bottom_box = widgets.HBox([btn_upload_building_types_and_cat_to_eep, chk_commit_upload_building_types_and_cat_to_eep])
        
        output =  widgets.VBox([content_box, lbl_spacer_update_metadata_type, bottom_box])
        
        
        
        #disables button after selected
        btn_choose_metadata_to_update.disabled = True
        rbtn_choose_metadata_to_update.disabled = True
        input_rbtn_AAA_meta_update.disabled = True
        
        
        
    #--------   
    
    elif rbtn_choose_metadata_to_update.value == 'Area':
        #output = str('Area chosen. No functionality here yet!')
        
        
        rbtn_choose_building_level_area = widgets.RadioButtons(
            options=['Total building area (distributed evely)', 'Floor areas (distributed evely)', 'Rooms (entered individually)'],
            value='Total building area (distributed evely)',
            description='Choose method to set area:',
            disabled=False,
            layout={'width':'350pt'},
            style={'description_width': '200px'}
        )

        
        output =  widgets.HBox([rbtn_choose_building_level_area, btn_choose_building_level_area])
        

        #disables button after selected
        btn_choose_metadata_to_update.disabled = True
        rbtn_choose_metadata_to_update.disabled = True
        input_rbtn_AAA_meta_update.disabled = True
    
    #---------
    
    
    elif rbtn_choose_metadata_to_update.value == 'Room Type':
        
        #empty lists to start extraction from lists
        data_list_for_room_type_update = []
        data_list_for_room_type_update_type = []
        
        #retrieving data from raw and eep-api and creating selection widgets
        room_type_update_meta_list = get_room_type_list_from_raw()
        
        try:
            room_list = get_room_list_from_eep_api(input_rbtn_AAA_meta_update.value)
            btn_add_type_to_room.disabled = False
            
        except:
            room_list = []
            btn_add_type_to_room.disabled = True
            print('No rooms found! Try creating physical building.')

        lbl_select_roomtype_list = widgets.Label(value='Room Types:')
        select_roomtype_list = widgets.Select(
            options=room_type_update_meta_list,
            layout={'width': '250px', 'height' : '800px'}
        )

        lbl_spacer_room_type_to_room = widgets.Label(value='', layout={'width':'50px'})

        lbl_select_rooms_list = widgets.Label(value='Rooms:')
        select_room_list = widgets.SelectMultiple(
            options=room_list,
            layout={'width': '800px', 'height': '800px'},
            disabled=False
        )

        output = widgets.HBox([widgets.VBox([lbl_select_roomtype_list, select_roomtype_list]),
                               lbl_spacer_room_type_to_room , widgets.VBox([lbl_select_rooms_list,
                                select_room_list, btn_add_type_to_room])])
        
        #disables button after selected
        btn_choose_metadata_to_update.disabled = True
        rbtn_choose_metadata_to_update.disabled = True
        input_rbtn_AAA_meta_update.disabled = True
    
    else:
        output = str('Nothing chosen')
        pass
    
    with output_choose_type_update_meta:
        output_choose_type_update_meta.clear_output()
        display(output)
        
btn_choose_metadata_to_update.on_click(choose_type_update_meta)

widgets.VBox([output_choose_type_update_meta])

In [35]:
output_area_widgets = widgets.Output()
btn_set_area = widgets.Button(description='Upload areas to EEP',layout={'width':'350pt'})
btn_set_area.style.button_color = 'lightgreen'

btn_choose_building_level_area = widgets.Button(description='Choose selected method',
                                                style= {'description_width': '200px'},
                                                layout={'width':'350pt'})

def create_area_widgets(l):
    
    global area_output
    
    if rbtn_choose_building_level_area.value == 'Total building area (distributed evely)':
        
        building_structure_dict = building_structure_client.list_metadata(AAA=AAA_meta_update_selected)
        
        area = building_structure_dict.get('area')
        
        total_area_widget = widgets.Text(
            value= str(area),   #input for existing value
            description='Total area for building:',
            disabled=False,
            layout={'width':'350pt'},
            style={'description_width': '200px'}
        )
        
        area_output = [[total_area_widget]]
        
    
    elif rbtn_choose_building_level_area.value == 'Floor areas (distributed evely)':
        
        #find floors with rooms
        building_structure_dict = building_structure_client.list(AAA=AAA_meta_update_selected)

        floor_list = []
        floor_no_list = []
        floor_area_list = []
        
        for k,v in building_structure_dict[AAA_meta_update_selected].items():
            if k != 'metadata':
                floor_no_list.append(k)
                room_list = []

                for k1, v1 in v.items():
                    if k1 != 'metadata':
                        room_list.append(k1)
                    if k1 == 'metadata':
                        floor_area_list.append(v1.get('area'))

                floor_list.append(room_list)
                

        #check if any floor has 0 rooms, then this floor is dropped
        list_index = 0
        drop_index_list = []
        for floor in floor_list:
            if len(floor) == 0:
                drop_index_list.append(list_index)

            list_index +=1

        #removes floors from list of floor_list
        new_floor_list = [ elem for elem in floor_list if len(elem) != 0]
        
        
        drop_flor_no_list = []
        for item in drop_index_list:
            drop_flor_no_list.append(floor_no_list[item])
            

        floor_no_w_rooms_list = [e for e in floor_no_list if e not in drop_flor_no_list]
        
        floor_area_w_rooms_list = [e for e in floor_area_list if e not in drop_flor_no_list]
        
        print(floor_area_w_rooms_list)
        
        floor_area_widg_list = []
        i = 0
        for floor in floor_no_w_rooms_list:
            
            floor_area_widg_list.append(widgets.Text(
                value=floor_area_w_rooms_list[i],  #input for existing value
                placeholder='area',
                description= 'Floor: ' + str(floor),
                disabled=False,
                layout={'width':'350pt'},
                style={'description_width': '200px'}
            ))
            
            i +=1
        
        area_output = [floor_area_widg_list]
        
        pass
    
    elif rbtn_choose_building_level_area.value == 'Rooms (entered individually)':
        
        
        building_structure_dict = building_structure_client.list(AAA=AAA_meta_update_selected)

        floor_list = []
        floor_no_list = []
        for k,v in building_structure_dict[AAA_meta_update_selected].items():
            if k != 'metadata':
                floor_no_list.append(k)
                room_list = []

                for k1, v1 in v.items():
                    if k1 != 'metadata':
                        room_data = {
                            'id' : str(k1),
                            'name' : v1.get('name'),
                            'desc' : v1.get('desc'),
                            'area' : v1.get('area')
                        }
                        
                        room_list.append(room_data)

                floor_list.append(room_list)
                
        #check if any floor has 0 rooms, then this floor is dropped
        list_index = 0
        drop_index_list = []
        for floor in floor_list:
            if len(floor) == 0:
                drop_index_list.append(list_index)

            list_index +=1

        #removes floors from list of floor_list
        new_floor_list = [ elem for elem in floor_list if len(elem) != 0]

        drop_flor_no_list = []
        for item in drop_index_list:
            drop_flor_no_list.append(floor_no_list[item])

        floor_no_w_rooms_list = [e for e in floor_no_list if e not in drop_flor_no_list]
        
        #--

        room_floor_area_widg_list = []
        
        i = 0
        for floor in new_floor_list:
            
            room_area_widg_list = []
            
            for room in floor:
                
                room_area_widg_list.append(widgets.Text(
                    value= room.get('area'),  #input for existing value
                    placeholder='m2',
                    description= 'Floor: ' + '"' + str(floor_no_w_rooms_list[i]) + '"' + ' - Room' + '(' + str(room.get('id')) +')' +': ' + str(room.get('name')) + ' - ' + str(room.get('desc')),
                    disabled=False,
                    layout={'width':'500pt'},
                    style={'description_width': '350px'}
                ))
            
            i+=1
            
            room_floor_area_widg_list.append(room_area_widg_list)
            
        area_output = room_floor_area_widg_list
    

    with output_area_widgets:
        output_area_widgets.clear_output()
        
        flat_list = [item for sublist in area_output for item in sublist]
        
        display(widgets.VBox(flat_list + [btn_set_area]))



btn_choose_building_level_area.on_click(create_area_widgets)
widgets.VBox([output_area_widgets])


In [36]:
def update_higher_level_area_metadata_from_rooms_and_commit(client, AAA_list = ['AAA']):
    building_structure_client = client
    
    for AAA in AAA_list:
        
        building_structure_dict = building_structure_client.list(AAA=AAA)
        
        area_floor_list = []
        
        for k,v in building_structure_dict[AAA].items():
            print('Floor ' + k)
            area_room_list = []
            if k != 'metadata':
            
                for k1, v1 in v.items():
                    #room level
                    #print(k1)
                    if k1 != 'metadata':
                        #print(k1)
                        #print(v1)
                        active_room_name = v1.get('name')
                        active_room_area = v1.get('area')
                        #print('Room_name: ' + str(active_room_name))
                        #print('Active_room_area: ' + str(active_room_area))

                        if active_room_area == '':
                            active_room_area = 0

                        area_room_list.append(active_room_area)
            
                #floor level

                #print(area_room_list)
                
                area_floor_list.append(sum(map(float, area_room_list)))

                floor_metadata_dict_old = building_structure_client.list_metadata(AAA=AAA, floor=k)
                
                if floor_metadata_dict_old:
                    pass
                else:
                    floor_metadata_dict_old = {}

                floor_metadata_dict = {
                    'AAA' : AAA,
                    'floor' : k,
                    'floor_type' : floor_metadata_dict_old.get('type'),
                    'category' : floor_metadata_dict_old.get('category'),
                    'area' : str(round(sum(map(float, area_room_list)),0)),
                    'energy_label' : floor_metadata_dict_old.get('energy_label'),
                    'opening_hours' : floor_metadata_dict_old.get('opening_hours'),
                    'metadata' : floor_metadata_dict_old.get('metadata',{})
                }

                #print(floor_metadata_dict)

                building_structure_client.add_metadata_floor(AAA = floor_metadata_dict.get('AAA'),
                                                             floor = floor_metadata_dict.get('floor'),
                                                             floor_type = floor_metadata_dict.get('floor_type'),
                                                             category = floor_metadata_dict.get('category'),
                                                             area = floor_metadata_dict.get('area'),
                                                             energy_label = floor_metadata_dict.get('energy_label'),
                                                             opening_hours = floor_metadata_dict.get('opening_hours'),
                                                             metadata = floor_metadata_dict.get('metadata'),
                                                             replace = True
                                                            )
        
        #sub_building level
        
        sub_building_metadata_dict_old = building_structure_client.list_metadata(AAA=AAA)
              
        if sub_building_metadata_dict_old:
                        
            #print(type(sub_building_metadata_dict_old))
            
            sub_building_metadata_dict = {
                
                'AAA' : AAA,
                'category' : sub_building_metadata_dict_old.get('category'),
                'building_type' : sub_building_metadata_dict_old.get('type'),
                'area' : str(round(sum(area_floor_list),1)),
                'energy_label' : sub_building_metadata_dict_old.get('energy_label'),
                'opening_hours' : sub_building_metadata_dict_old.get('opening_hours'),
                'metadata' : sub_building_metadata_dict_old.get('metadata',{})
            }
        
        else:
            
            sub_building_metadata_dict = {
                
                'AAA' : AAA,
                'category' : '',
                'building_type' : '',
                'area' : str(round(sum(area_floor_list),0)),
                'energy_label' : '',
                'opening_hours' : '',
                'metadata' : {}
            }
            
        #print(sub_building_metadata_dict)

        building_structure_client.add_metadata_sub_building(AAA = AAA,
                                                            building_type = sub_building_metadata_dict.get('building_type'),
                                                            category = sub_building_metadata_dict.get('category'),
                                                            area = sub_building_metadata_dict.get('area'),
                                                            energy_label = sub_building_metadata_dict.get('energy_label'),
                                                            opening_hours = sub_building_metadata_dict.get('opening_hours'),
                                                            metadata = sub_building_metadata_dict.get('metadata'),
                                                            replace = True)
    
    building_structure_client.commit()
    
    #print(building_structure_client.list_metadata())

    #return 
    

In [37]:
def generate_building_areas_sub_building(total_area, AAA):
    
    #all higher level building structure areas are a sum function of all items on the room level
    

        # set area only from total area of building, distribute evenly over floors and all rooms.
        
        #get all floors
        # devide total area on nmber of rooms
        
    print('total_area: ' + str(total_area))
        
    building_structure_dict = building_structure_client.list(AAA=AAA)
    
    floor_list = []
    floor_no_list = []
    for k,v in building_structure_dict[AAA].items():
        if k != 'metadata':
            floor_no_list.append(k)
            room_list = []
            
            for k1, v1 in v.items():
                if k1 != 'metadata':
                    room_list.append(k1)

            floor_list.append(room_list)
   
   
    
    print('Total floors: ' + str(len(floor_list)))

    #check if any floor has 0 rooms, then this floor is dropped
    list_index = 0
    drop_index_list = []
    for floor in floor_list:
        if len(floor) == 0:
            drop_index_list.append(list_index)
        
        list_index +=1
    
    #removes floors from list of floor_list
    new_floor_list = [ elem for elem in floor_list if len(elem) != 0]
    
    drop_flor_no_list = []
    for item in drop_index_list:
        drop_flor_no_list.append(floor_no_list[item])
    
    #print(drop_flor_no_list)
    
    
    new_floor_no_list = [e for e in floor_no_list if e not in drop_flor_no_list]
    
    print('Floors with rooms: ' + str(new_floor_no_list))
    
    print('Total floors with rooms: ' + str(len(new_floor_list)))
    
    floor_area_avg = float(total_area) / len(new_floor_list)
    
    print('Snitt areal pr etg: ' + str(floor_area_avg))
    
    
    i = 0
    for floor in new_floor_list:
        tot_rooms_in_floor = len(floor)
        
        print('Floor: ' + str(new_floor_no_list[i]))
        
        print('tot_rooms_in_floor: ' + str(tot_rooms_in_floor))
        
        for room in floor:
            #print(room)
            #print(str(floor_area_avg/tot_rooms_in_floor))
            
            building_structure_client.update_room(
                room_id = str(room),
                AAA = str(AAA),
                floor = str(new_floor_no_list[i]),
                area = str(round(floor_area_avg/tot_rooms_in_floor,2))
            )

        
        i +=1

        
       
    
    return floor_list, floor_no_list, new_floor_no_list

In [38]:
def generate_building_areas_floor(total_area_floors_list, AAA):
    
    #all higher level building structure areas are a sum function of all items on the room level
    

    # set area from list of total area pr floor, distribute evenly over floors and all rooms.

    #get all floors and devide the floor_area over rooms
    # devide total area from eac floor evenly on number of rooms
        
    print('total_area: ' + str(sum(total_area_floors_list)))
        
    building_structure_dict = building_structure_client.list(AAA=AAA)
    
    floor_list = []
    floor_no_list = []
    for k,v in building_structure_dict[AAA].items():
        if k != 'metadata':
            floor_no_list.append(k)
            room_list = []
            
            for k1, v1 in v.items():
                if k1 != 'metadata':
                    room_list.append(k1)

            floor_list.append(room_list)
   
       
    print('Total floors: ' + str(len(floor_list)))

    #check if any floor has 0 rooms, then this floor is dropped
    list_index = 0
    drop_index_list = []
    for floor in floor_list:
        if len(floor) == 0:
            drop_index_list.append(list_index)
        
        list_index +=1
    
    #removes floors from list of floor_list
    new_floor_list = [ elem for elem in floor_list if len(elem) != 0]
    
    drop_flor_no_list = []
    for item in drop_index_list:
        drop_flor_no_list.append(floor_no_list[item])
    
    #print(drop_flor_no_list)
    
    
    new_floor_no_list = [e for e in floor_no_list if e not in drop_flor_no_list]
    
    print('Floors with rooms: ' + str(new_floor_no_list))
    
    print('Total floors with rooms: ' + str(len(new_floor_list)))
    
    print('------')
    
    #check if length of input total_area_floors_list matches the number of floors in selected subbuilding
    
    if len(total_area_floors_list) == len(new_floor_list):
    
        i = 0
        for floor in new_floor_list:
            tot_rooms_in_floor = len(floor)

            print('Floor: ' + str(new_floor_no_list[i]))

            print('tot_rooms_in_floor: ' + str(tot_rooms_in_floor))
            print('avg room area in floor: ' + str(total_area_floors_list[i]/tot_rooms_in_floor))

            for room in floor:
                #print(room)
                #print(str(floor_area_avg/tot_rooms_in_floor))

                building_structure_client.update_room(
                    room_id = str(room),
                    AAA = str(AAA),
                    floor = str(new_floor_no_list[i]),
                    area = str(round(total_area_floors_list[i]/tot_rooms_in_floor,2))
                )


            i +=1
    else:
        print('ERROR! Missmatch between numder of input floor areas and the number of found floors with rooms.')
        print('No area updated. Exiting...')
        print('')

       
    
    return floor_list, floor_no_list, new_floor_no_list

In [39]:
def generate_building_areas_rooms(total_area_rooms_list, AAA):
    
    for floor in area_output:
        for room in floor:
            
            room_id = find_between(room.description, '(',')' )
            floor_no = find_between(room.description, '"','"' )
            area = room.value
            
            if area == '':
                area = 0
            
            building_structure_client.update_room(
                room_id = str(room_id),
                AAA = str(AAA_meta_update_selected),
                floor = str(floor_no),
                area = str(area)
            )
    
    

In [40]:
output_set_area = widgets.Output()

def upload_building_areas_room_to_eepa(o):
    
    f2.value = 0
    f2.max = 3
   
    #total given
    if rbtn_choose_building_level_area.value == 'Total building area (distributed evely)':
        
        total_area = area_output[0][0].value
        
        f2.value += 1
        
        generate_building_areas_sub_building(total_area, AAA=str(AAA_meta_update_selected))
        
    
    #each floor given
    if rbtn_choose_building_level_area.value == 'Floor areas (distributed evely)':
        total_area_floors_list = []
        
        for floor in area_output[0]:
            total_area_floors_list.append(float(floor.value))            
        
        f2.value += 1
        
        generate_building_areas_floor(total_area_floors_list, AAA=str(AAA_meta_update_selected))
    
    
    #all romms given
    if rbtn_choose_building_level_area.value == 'Rooms (entered individually)':
        
        total_area_rooms_list = area_output
        
        f2.value += 1
        
        generate_building_areas_rooms(total_area_rooms_list, AAA=str(AAA_meta_update_selected))
    
    
    
    #buttom up approach updating
    
    f2.value += 1
    
    update_higher_level_area_metadata_from_rooms_and_commit(client = building_structure_client, AAA_list = [str(AAA_meta_update_selected)])
    
    f2.value += 1
    
    with output_set_area:
        output_set_area.clear_output()
        
        display(print('Done! Area updated in EEP'))
        
btn_set_area.on_click(upload_building_areas_room_to_eepa)

In [41]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [42]:
output_add_type_to_room_and_remove_item_from_list = widgets.Output()
output_add_type_to_room_and_remove_item_from_list_text = widgets.Textarea(disabled=True, layout = {'width': '800pt', 'height': '300pt'})

btn_upload_types_to_eep = widgets.Button(description='Update building structure with room types', layout = {'width': '300pt'})
btn_upload_types_to_eep.style.button_color = 'lightgreen'

data_list_for_room_type_update = []
data_list_for_room_type_update_type = []

def add_type_to_room_and_remove_item_from_list(i):
    
    global data_list_for_room_type_update, data_list_for_room_type_update_type
    
    active_room_list = select_room_list.value
    
    if len(active_room_list) > 0:
    
        data_list = get_values_from_selected_room_list(select_room_list.value)
        data_list_for_room_type_update.append(data_list)

        data_list_for_room_type_update_type.append(str(select_roomtype_list.value))

        output = data_list_for_room_type_update

        output_add_type_to_room_and_remove_item_from_list_text.value = str(output)

        #removing items from list of all rooms in widget, that have been assigned room type
        for item in active_room_list:
            try:
                room_list.remove(item)
            except:
                print('Error removing item from list of rooms...')

        #updating widget content
        select_room_list.options = room_list
    
   
    with output_add_type_to_room_and_remove_item_from_list:
        output_add_type_to_room_and_remove_item_from_list.clear_output()
        display(widgets.VBox([output_add_type_to_room_and_remove_item_from_list_text, btn_upload_types_to_eep]))
        
btn_add_type_to_room.on_click(add_type_to_room_and_remove_item_from_list)

widgets.VBox([output_add_type_to_room_and_remove_item_from_list])

In [43]:
output_uplad_all_types_to_eep = widgets.Output()

f2_max_count = 1

f2 = widgets.IntProgress(min=0, max=f2_max_count) # instantiate the bar

def upload_all_types_to_eep(r):
    
    f2.value = 0
   
    f2.max = len(data_list_for_room_type_update_type) + 2
    
    for item in data_list_for_room_type_update_type:
        for room in data_list_for_room_type_update[data_list_for_room_type_update_type.index(item)]:
            
            building_structure_client.update_room(room_id= room[2], AAA=room[0], floor=room[1], room_type=str(item))
        
        f2.value +=1
    
    with output_uplad_all_types_to_eep:
        output_uplad_all_types_to_eep.clear_output()
        
        building_structure_client.dump_building_structure()
        f2.value +=1
   
        building_structure_client.commit()
        f2.value +=1

btn_upload_types_to_eep.on_click(upload_all_types_to_eep)

widgets.VBox([f2, output_uplad_all_types_to_eep])

In [44]:
def add_types_to_floors(p):
    
    global building_type_list_selected, building_cat_list_selected
    
    for floor in input_floors_metadata_multiselect.value:
#         building_type_str = ""
#         building_cath_str = ""
        
        building_type_list_selected = building_type_widget_list[widget_building_type.selected_index].value
        
        building_cat_list_selected = building_cat_list[widget_building_type.selected_index]
        
        update_floor_metadata(floor_metadata_dict_list, floor_no=floor,
                              building_type=building_type_list_selected,
                              building_category=building_cat_list_selected)
        
        #floors_type_metadata.options = floor_metadata_dict_list
    
btn_add_types_to_floors.on_click(add_types_to_floors)


In [45]:
def clear_types_to_floors(q):
    for floor in input_floors_metadata_multiselect.value:
        clear_floor_metadata(floors_metadata_list=floor_metadata_dict_list, floor_no=floor)

    floors_type_metadata.options = floor_metadata_dict_list
    
btn_clear_types_to_floors.on_click(clear_types_to_floors)

In [46]:
# floor_metadata = building_structure_client.list_metadata(AAA='FV50', floor='005')
# floor_metadata
#floor_metadata_dict_list

In [47]:
def upload_building_types_and_cat_to_eep_metadata(s):
    
    for floor_item in floor_metadata_dict_list:
        
        floor = floor_item.get('floor_no')
        
        #read metadata for selected flor/item
        floor_metadata = building_structure_client.list_metadata(AAA=input_rbtn_AAA_meta_update.value,
                                                                 floor=floor)
        
        print('Read old metadata from floor: ' + str(floor))
        #print(floor_metadata)
        
        
        if floor_metadata == None:
            floor_metadata = {}
        
        
        building_structure_client.add_metadata_floor(AAA = (input_rbtn_AAA_meta_update.value),
                                                    floor = str(floor),
                                                    floor_type = str(floor_item.get('building_type')),
                                                    category = str(floor_item.get('building_category')),
                                                    area = str(floor_metadata.get('area')),
                                                    energy_label = str(floor_metadata.get('energy_label')),
                                                    opening_hours = str(floor_metadata.get('opening_hours')),
                                                    metadata = floor_metadata.get('metadata', {}),
                                                    replace = True)
    
    #upload to eep
    if chk_commit_upload_building_types_and_cat_to_eep.value == False:
        
        building_structure_client.commit()
    
    
    with output_upload_building_types_and_cat_to_eep:
        output_upload_building_types_and_cat_to_eep.clear_output()
        
        building_structure_client.dump_building_structure()
        
        if chk_commit_upload_building_types_and_cat_to_eep.value == False:
            display(print('Upladed to EEP!'))
        else:
            display(print('Test print out!'))

output_upload_building_types_and_cat_to_eep = widgets.Output()
btn_upload_building_types_and_cat_to_eep = widgets.Button(description='UPLOAD Floor types and categories to EEP', layout = {'width':'350px'})
btn_upload_building_types_and_cat_to_eep.style.button_color = 'lightgreen'
btn_upload_building_types_and_cat_to_eep.on_click(upload_building_types_and_cat_to_eep_metadata)

widgets.VBox([output_upload_building_types_and_cat_to_eep])

***

[HOME](https://eepcloud.ngrok.io)

In [48]:
# building_structure_dict = building_structure_client.list(AAA=AAA_meta_update_selected)
# floor_list = []
# floor_no_list = []
# for k,v in building_structure_dict[AAA_meta_update_selected].items():
#     if k != 'metadata':
#         floor_no_list.append(k)
        
#         print(k)
        
#         room_list = []

#         for k1, v1 in v.items():
#             if k1 != 'metadata':
#                 #print(k1)
#                 print(v1.get('name'))
#                 print(v1.get('desc'))
#                 print(v1.get('area'))
#                 room_list.append(k1)

#         floor_list.append(room_list)

In [49]:
# floor_index_list = 0

# for floor in area_output:
#     print(floor_no_w_rooms_list[floor_index_list])
#     for room in floor:
#         print('Descr: ' + str(room.description))
#         print('area: ' + str(room.value))
    
#     print('-')
#     floor_index_list +=1
    

In [50]:
#Test zone

In [51]:
# building_structure_client = PhysicalBuildingAPI(cdf_project="energima",
#                                                 root_external_id='AH_1366_Fornebuveien_50',
#                                                 dryrun=True)
# building_structure_dict = building_structure_client.list(AAA='FV50')
# building_structure_dict

In [52]:
#update_higher_level_area_metadata_from_rooms(client=building_structure_client, AAA_list = ['FV50'])

In [53]:
# building_structure_client = PhysicalBuildingAPI(cdf_project="energima",
#                                                 root_external_id='AH_0154_Dronningens_gate_40',
#                                                 dryrun=False)
# building_structure_dict = building_structure_client.list(AAA='AAA')
# building_structure_dict

In [54]:
#floor_list, floor_no_list, new_floor_no_list = generate_building_areas_sub_building(5000, 'AAA', dump_results=True)

In [55]:
#floor_list, floor_no_list, new_floor_no_list = generate_building_areas_floor([500,1000,2000,3000], AAA='AAA', dump_results = True)

In [56]:
#update_higher_level_area_metadata_from_rooms(client = building_structure_client, AAA_list = ['AAA'])

In [57]:
#building_structure_client.commit()

In [58]:
#building_structure_client.dump_building_structure()

In [59]:
#     # find all floors in chosen sub-building
#     floor_area_list = []
#     for floor in floors_list:
#     # find all rooms in chosen sub-building, then iterate over each room, find its metadata 'area', then sum up to get total area for selected floor in metadata 'area'
    
#         floor_area_list.append(floor_area)
    
#         # set sub_building area as metadata
    
#     sub_building_area = float(sum(floor_area_list))
    
#     building_structure_client.metadata(metadata = floor_metadata,
#                                        AAA=input_rbtn_AAA_meta_update.value,
#                                        floor=floor)
    

    
#     add_metadata_building(AAA: str,
#                           building_type: str = None,
#                           category: str = None,
#                           area: str = None,
#                           energy_label: str = None,
#                           opening_hours: str = None)    

In [60]:
# floor_index = 0
# pop_list = []
# for room_list in floor_list:
#     if len(room_list) == 0:
#         print('empty floor ' + str(floor_index))
#         pop_list.append(floor_index)
#     floor_index +=1

# print(pop_list)

# for item in pop_list:
#     del floor_list.pop(item)
    

In [61]:
# building_structure_dict = building_structure_client.list(AAA='FV50')

# floor_list = []
# for k,v in building_structure_dict['FV50'].items():
#     if k != 'metadata':

#         room_list = []
#         for k1, v1 in v.items():
#             if k1 != 'metadata':
#                 room_list.append(k1)

#     floor_list.append(room_list)